In [4]:
import sys
import datetime
import psycopg2 as ppg
import pandas as pd
import numpy as np

con = ppg.connect("dbname=tweets user=patrick")
print con

usa = (-125.6791025,25.4180700649,-66.885417,
                           49.3284551525)
la = ( -119.578941, 32.799580, -114.130814, 35.809120)
    
def in_place(pts, box, long_field='tweet_long', lat_field='tweet_lat'):
    return pd.Series((pts[long_field] > box[0]) & 
                     (pts[lat_field] > box[1]) &
                     (pts[long_field] < box[2]) &
                     (pts[lat_field] < box[3]))

with con.cursor() as cur:
    cur.execute("SELECT tw.tweet_id, tw.user_id, tw.lat as tweet_lat, tw.long as tweet_long, "
                "tw.created_at, user_locs.followers_count,user_locs.friends_count, "
                "user_locs.place as user_location, plcs.user_lat, "
                "plcs.user_long "
                "FROM tweets as tw, (SELECT user_id, place, friends_count, followers_count "
                "FROM users) as user_locs, "
                "(SELECT place, coded_lat as user_lat, coded_long as user_long FROM "
                "places) as plcs "
                "WHERE tw.user_id = user_locs.user_id and user_locs.place != 'None' "
                "and plcs.place = user_locs.place AND plcs.user_lat != -500 AND "
                "plcs.user_long != 500 AND (user_locs.place ~* '[A-Z\s]+,\s*[A-Z]+' OR "
                "user_locs.place ~* '(New York City|NYC|LA|Los Angeles|Chicago|DC|SF|San Francisco|"
                "Detroit|Houston|Dallas|Atlanta|Philly|Philadelphia|Phoenix|San Antonio| "
                "San Jose|Austin|Jacksonville|Indianapolis|Denver)') AND "
                "tw.text !~* '#insurancejobs|#jobs?|#hiring|#tweetmyjobs';")
    
    geotags = pd.DataFrame(cur.fetchall(), 
                       columns=[c[0] for c in cur.description])
    
geotags['in_us'] = in_place(geotags[['tweet_long','tweet_lat']], usa)
geotags['in_la'] = in_place(geotags[['tweet_long','tweet_lat']], la)

geotags= geotags.merge(geotags.groupby('user_id').apply(lambda x: pd.Series(x.shape[0], index=['num_tweets_user'])),
                    left_on='user_id', right_index=True, how='left')

# drop certain users
geotags = geotags[(geotags.followers_count < 2000) & 
                  (geotags.friends_count < 1800) &
                  (geotags.num_tweets_user < 200) &
                  (geotags.num_tweets_user >= 3) &
                  (geotags.friends_count / geotags.followers_count >= 0.25)]

# look at distance between user location and tweet location

geotags['tweet_distance'] = np.sqrt((geotags.user_lat - geotags.tweet_lat) ** 2 + 
                                    (geotags.user_long - geotags.tweet_long) ** 2)
geotags['on_road'] = geotags['tweet_distance'] > 0.5
geotags['time_of_day'] =  geotags.created_at.map(lambda x: x.hour)

# get number of users w/ any on-road tweets, other per-user stats
users_on_road=geotags.groupby('user_id').apply(lambda x: pd.Series([x.on_road.any(), x.on_road.sum()],
         index=['any_on_road','num_on_road']))
geotags = geotags.merge(users_on_road, left_on='user_id', right_index=True)

# get locations for tweets in LA
import re
# get number of distinct users
print "Number of users: ", geotags[geotags.in_la].user_id.drop_duplicates().shape[0]
# their locations
#print geotags[geotags.in_la][['user_id','user_location']].drop_duplicates().user_location
def normalizer(txt):
    txt = txt.strip()
    txt = txt.lower()
    
    txt = re.sub(ur"[,.]", u" ", txt,  re.UNICODE)
    txt = re.sub(ur"\s+", u" ", txt, re.UNICODE)
    txt = re.sub(ur"cali[^\s]+", u"ca", txt, re.UNICODE | re.IGNORECASE)
    return txt
#print geotags[geotags.in_la][['user_id','user_location']].drop_duplicates().user_location.map(normalizer).value_counts().index.tolist()

# get places
with con.cursor() as cur:
    cur.execute("SELECT user_id, place, coded_lat, coded_long FROM users "
                "JOIN (SELECT place, coded_lat, coded_long FROM places) AS places USING (place)")
    places_df = pd.DataFrame(cur.fetchall(), columns = [c[0] for c in cur.description])
places_df['places_norm'] = places_df.place.map(lambda x: x.decode('utf-8')).map(normalizer)
places_df = places_df.merge(pd.DataFrame(geotags[geotags.in_la][['user_id','user_location']]. \
                                         drop_duplicates().user_location.map(lambda x: x.decode('utf-8')).
                                         map(normalizer)),
               left_on='places_norm',right_on='user_location')

places_df['in_la'] = places_df['coded_long'].map(lambda x: (x > la[0]) & (x < la[2])) & \
                     places_df['coded_lat'].map(lambda x: (x > la[1]) & (x < la[3])) | \
                    (places_df.places_norm == "ca") | (places_df.places_norm == "la") | \
                    (places_df.places_norm == 'san diego ca')
        
# limit to ppl who tweet in LA
geotags_la = geotags[geotags.in_la].merge(places_df, on='user_id', suffixes = ('_tweet','_user')).drop_duplicates()

with con.cursor() as cur:
    cur.execute("SELECT tweet_id, text, user_id, tokens FROM tweets")
    tweets_df = pd.DataFrame(cur.fetchall(), columns=[c[0] for c in cur.description])
la_tourist_tweets = geotags_la[geotags_la.in_la_user==False].merge(tweets_df, on='tweet_id')
la_native_tweets = geotags_la[geotags_la.in_la_user==True].merge(tweets_df, on='tweet_id')


<connection object at 0x7f7719634050; dsn: 'dbname=tweets user=patrick', closed: 0>
Number of users:  1375


In [30]:
# language modeling
from nltk.util import ngrams

def normalizer(txt):
    txt = txt.strip()
    txt = txt.lower()
    
    txt = re.sub(ur"[,.]", u" ", txt,  re.UNICODE)
    txt = re.sub(ur"\s+", u" ", txt, re.UNICODE)
    txt = re.sub(ur"cali[^\s]+", u"ca", txt, re.UNICODE | re.IGNORECASE)
    return txt

def add_boundaries(tkns):
    return ["#"] + tkns + ["#"]

native_bigrams = ngrams([tkn for tkns in la_native_tweets.tokens.dropna().map(
            lambda x: x.decode('utf-8')).map(normalizer). \
                             map(unicode.split).map(add_boundaries) for tkn in tkns], 2)
nbg = pd.DataFrame(native_bigrams)
nbg['ct'] = 1
nbg_df = nbg.groupby([0,1]).agg(['count', lambda x: float(x.count()) / float(nbg.ct.sum())])

tourist_bigrams = ngrams([tkn for tkns in la_tourist_tweets.tokens.dropna().map(
            lambda x: x.decode('utf-8')).map(normalizer). \
                             map(unicode.split).map(add_boundaries) for tkn in tkns], 2)
tbg = pd.DataFrame(tourist_bigrams)
tbg['ct'] = 1
tbg_df = tbg.groupby([0,1]).agg(['count', lambda x: float(x.count()) / float(tbg.ct.sum())])

bigram_models = pd.concat([tbg_df, nbg_df], axis=0, keys=['tourist','not tourist'])

In [31]:
bigram_models

ct          
                                                    count  <lambda>
        0                     1                                    
!       #                     #                        14  0.000852
        #6thstviaduct         #6thstviaduct             1  0.000061
        #actress              #actress                  1  0.000061
        #bnsbrewing           #bnsbrewing               1  0.000061
        #ca                   #ca                       1  0.000061
        #chips                #chips                    1  0.000061
        #darpadrc             #darpadrc                 1  0.000061
        #disneyland           #disneyland               1  0.000061
        #dodgerstadium        #dodgerstadium            1  0.000061
        #dope                 #dope                     1  0.000061
        #e3                   #e3                       1  0.000061
        #endeavor             #endeavor                 1  0.000061
        #fantasyfactory       #fantasyfactory           1  0.000061
        #getsthehose          #getsthehose              1  0.000061
        #hallelucci           #hallelucci               1  0.000061
        #haveaseat            #haveaseat                1  0.000061
        #heedmagazine         #heedmagazine             1  0.000061
        #insidious3           #insidious3               1  0.000061
        #lafilmfest           #lafilmfest               1  0.000061
        #letgola              #letgola                  1  0.000061
        #love                 #love                     1  0.000061
        #madebylaurenb        #madebylaurenb            1  0.000061
        #mangos               #mangos                   1  0.000061
        #mels                 #mels                     1  0.000061
        #momsdriving          #momsdriving              1  0.000061
        #nomnom               #nomnom                   1  0.000061
        #o                    #o                        1  0.000061
        #onthemove            #onthemove                1  0.000061
        #platinumblonde       #platinumblonde           1  0.000061
        #scenery              #scenery                  1  0.000061
...                                                   ...       ...
🙌🏾      great                 great                     1  0.000010
        💯😭😂                   💯😭😂                       1  0.000010
🙌🏾😚🙊🙈🙉💋 @bythewayfilm         @bythewayfilm             1  0.000010
🙌🏿      (                     (                         1  0.000010
        @                     @                         1  0.000010
🙌👌✨👏    ive                   ive                       1  0.000010
🙌🙌👌🙏    #                     #                         1  0.000010
🙏       #                     #                         1  0.000010
        thank                 thank                     2  0.000020
🙏thank  you                   you                       1  0.000010
🙏🏻      #practicemakesperfect #practicemakesperfect     1  0.000010
🙏🏼      #                     #                         1  0.000010
        stop                  stop                      1  0.000010
🙏🏽🎶     @                     @                         1  0.000010
🙏💕      #forevergrateful      #forevergrateful          1  0.000010
🙏💯      #istayfit             #istayfit                 1  0.000010
🚀🌛      @8biteric             @8biteric                 1  0.000010
🚎       trailer               trailer                   4  0.000040
🚘✨🙌     #                     #                         1  0.000010
🚚       video                 video                     1  0.000010
🚢       #                     #                         1  0.000010
🚦🚘      @                     @                         1  0.000010
🚧       #fearfactor           #fearfactor               2  0.000020
🚪       is                    is                        1  0.000010
🚫🍞😷     @                     @                         1  0.000010
🚬🚬🚬🚬🚬   @la_dollhouse         @la_dollhouse